# The cutting stock problem
Consider the Cutting Stock problem introduced in the Classical Problems library. The scope of this task is to compare the two formulations provided for the problem.
The class provided below generates random instances of the problem.
Particularly, run the following experiment
- Generate 10 random instances of the problem with 7 types of small rolls each (`n_types`)
- For each instance solve the LP relaxation of the two formulations and record their objectives (LP bounds)
- For each instance solve both integer formulations and recall solution times and optimality gaps
- Using basic statistics, draw conclusions regarding the quality of the two formulations.

In [1]:
import random as r
class CuttingStockProblem:
    
    def __init__(self,n_types, seed:int=1):
        r.seed(seed)
        self.n_types = n_types
        self.width_large_rolls = r.randint(5,10)
        self.width_small_rolls = [1.0 + 2 * r.random() for i in range(self.n_types)]
        self.demand = [5 + r.randint(0,15) for i in range(self.n_types)]
    
    def get_max_number_of_large_rolls(self):
        '''
        Returns an upper bound on the number of large rolls used.
        It assumes that each small roll is cut from a different large roll,
        therefore, the upper bound is equal to the sum of the demands.
        :return:int
        '''
        return sum(self.demand)


    def get_feasible_patterns(self):

        # First we calculate how many of each width we can cut from a large roll
        max_rolls = [int((self.width_large_rolls/self.width_small_rolls[i])//1) for i in range(self.n_types)]

        # Now, we generate all possible patterns, and then we
        # check which ones are feasible. All
        # possible patterns are given by the Cartesian product of
        # the integers up to max_rolls.
        # That is, if we can cut
        # -- up to 2 of width w1, that is 0, 1 or 2
        # -- up to 2 of width w2, that is 0, 2 or 2
        # -- up to 3 of width w3, that is 0, 1, 2 or 3
        # all the possible patters are given by the Cartesian product
        # of the vectors[0, 1, 2]x[0, 1, 2]x[0, 1, 2, 3]
        vectors = {}
        for i in range(self.n_types):
            vectors[i] = [j for j in range(max_rolls[i] + 1)]

        patterns = None
        for i in vectors:
            if patterns == None:
                patterns = self.cartesian_product_of_one(vectors[i])
            else:
                patterns = self.cartesian_product_of_two(patterns,vectors[i])

        # Now, of the cartesian products, we discard the elements which violate the large roll width
        infeasible_patterns = []
        for p in patterns:
            cut_width = sum([self.width_small_rolls[i] * p[i] for i in range(self.n_types)])
            if cut_width > self.width_large_rolls:
                infeasible_patterns.append(p)
        for p in infeasible_patterns:
            patterns.remove(p)

        return patterns

    def cartesian_product_of_one(self,a:list):
        return [(i,) for i in a]

    def cartesian_product_of_two(self,a:list,b:list) -> list:
        elements = []
        for i in a:
            for j in b:
                elements.append(i+(j,))
        return(elements)


# Solution